## 필요 모듈 불러오기

In [23]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import glob
from PIL import Image
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D,Dense,Flatten,Dropout,MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

In [15]:
len(glob.glob('/content/drive/MyDrive/안봐도보이조_project/train_10/*/*'))

10529

## 이미지 생성기 선언

In [24]:
train_datagen = ImageDataGenerator( 
        rescale=1./255,         # 픽셀 값을 0~1 범위로 변환
        rotation_range=40,      # 40도까지 회전
        width_shift_range=0.2,  # 20%까지 좌우 이동
        height_shift_range=0.2, # 20%까지 상하 이동
        shear_range=0.2,        # 20%까지 기울임
        zoom_range=0.2,         # 20%까지 확대
        horizontal_flip=True,   # 좌우 뒤집기
    )
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/안봐도보이조_project/train_10',
    batch_size=64,      
    class_mode='categorical', 
    target_size=(64, 64)) 
valid_datagen = ImageDataGenerator(rescale=1./255) # 스케일링만 진행
valid_generator = valid_datagen.flow_from_directory(
    '/content/drive/MyDrive/안봐도보이조_project/test_10',
    batch_size=8,      
    class_mode='categorical', 
    target_size=(64, 64))

Found 10529 images belonging to 10 classes.
Found 500 images belonging to 10 classes.


## 모델 생성 및 학습
- ResNet50 모델 사용
- early_stopping, modelcheckpoint 적용

In [26]:
resnet50 = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(64 ,64 ,3))

resnet50.trainable = False
model = Sequential()
model.add(resnet50)
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2, 2, 2048)        23587712  
                                                                 
 flatten_3 (Flatten)         (None, 8192)              0         
                                                                 
 dense_14 (Dense)            (None, 10)                81930     
                                                                 
Total params: 23,669,642
Trainable params: 81,930
Non-trainable params: 23,587,712
_________________________________________________________________


In [27]:
model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [28]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)

In [29]:
model_path = '/content/drive/MyDrive/안봐도보이조_project/김강산/model/RedNet50.hdf5'

checkpointer = ModelCheckpoint(filepath=model_path, monitor='val_loss', 
                               verbose=1, save_best_only=True)

In [ ]:
history = model.fit(train_generator,
                    batch_size=64,
                    steps_per_epoch=10529//64, 
                    epochs=1000, 
                    validation_data=valid_generator,
                    validation_batch_size=8,
                    validation_steps=500//8,
                    callbacks=[early_stopping_callback, checkpointer])

Epoch 1/1000
164/164 [==============================] - ETA: 0s - loss: 2.1150 - accuracy: 0.2239
Epoch 1: val_loss improved from inf to 2.08096, saving model to /content/drive/MyDrive/안봐도보이조_project/김강산/model/RedNet50.hdf5
164/164 [==============================] - 70s 424ms/step - loss: 2.1150 - accuracy: 0.2239 - val_loss: 2.0810 - val_accuracy: 0.2460
Epoch 2/1000
164/164 [==============================] - ETA: 0s - loss: 2.0526 - accuracy: 0.2540
Epoch 2: val_loss improved from 2.08096 to 2.01512, saving model to /content/drive/MyDrive/안봐도보이조_project/김강산/model/RedNet50.hdf5
164/164 [==============================] - 58s 352ms/step - loss: 2.0526 - accuracy: 0.2540 - val_loss: 2.0151 - val_accuracy: 0.2883
Epoch 3/1000
164/164 [==============================] - ETA: 0s - loss: 2.0051 - accuracy: 0.2762
Epoch 3: val_loss improved from 2.01512 to 1.94955, saving model to /content/drive/MyDrive/안봐도보이조_project/김강산/model/RedNet50.hdf5
164/164 [====